In [ ]:
# Automatically gets job_id from URL - NO MANUAL INPUT!
display(Javascript("""
    const params = new URLSearchParams(window.location.hash.substring(1));
    const job_id = params.get('job_id') || '';
    const api_base = params.get('api_base') || 'http://localhost:5000';

    kernel.execute(`job_id = "${job_id}"; api_base = "${api_base}"`);
"""))

In [ ]:
response = requests.get(f"{api_base}/api/s2dr3/jobs/{job_id}")
job = response.json()

lon = job['lon']
lat = job['lat']
date = job['date']
farm = job['farm']

In [ ]:
wheel_url = "https://storage.googleapis.com/0x7ff601307fa5/s2dr3-20250905.1-cp312-cp312-linux_x86_64.whl"
!pip install -q {wheel_url}

In [ ]:
import s2dr3.inferutils

# THIS IS THE KEY LINE - Enhances 10m → 1m!
lonlat = (lon, lat)
s2dr3.inferutils.test(lonlat, date)

# Generates 3 enhanced images:
# - MS (Multi-Spectral): 10 bands at 1m
# - NDVI: Vegetation index at 1m
# - TCI (True Color): RGB at 1m

In [ ]:
import rasterio
from rasterio.mask import mask

def clip_to_farm(in_file, out_file, farm_geom):
    with rasterio.open(in_file) as src:
        out_image, out_transform = mask(src, [farm_geom], crop=True)
        # Save only your farm area
        with rasterio.open(out_file, "w", **meta) as dest:
            dest.write(out_image)

# Clip all 3 images
clip_to_farm(ms_file, "ms_clipped.tif", farm)
clip_to_farm(ndvi_file, "ndvi_clipped.tif", farm)
clip_to_farm(tci_file, "tci_clipped.tif", farm)

In [ ]:
upload_url = f"{api_base}/api/s2dr3/jobs/{job_id}/upload"
headers = {"Authorization": f"Bearer {upload_token}"}

files = {
    'ms': open('ms_clipped.tif', 'rb'),
    'ndvi': open('ndvi_clipped.tif', 'rb'),
    'tci': open('tci_clipped.tif', 'rb')
}

response = requests.post(upload_url, headers=headers, files=files)
# ✅ Enhanced images now available in DHARTI!